In [ ]:
# default_exp funcs.emitter_io

In [ ]:
%load_ext autoreload
%autoreload 2

# Functions for working with emitter DFs

In [ ]:
#export
from decode_fish.imports import *

In [ ]:
#export
def shift_df(df, shift=[0.,0.,0.]):
    
    df_corr = df.copy()
    df_corr['x'] += shift[0]
    df_corr['y'] += shift[1]
    df_corr['z'] += shift[2]
    return df_corr

def percentile_filter(df, perc, key='comb_sig', return_low=True):
    if perc >= 100:
        return df
    filt_val = np.percentile(df[key], perc)
    if return_low:
        return df[df[key] < filt_val]
    else:
        return df[df[key] > filt_val]


def sig_filt(df, perc = 90, return_low=True):
    if perc >= 100:
        return df
    filt_val = np.percentile(df['comb_sig'], perc)
    if return_low:
        return df[df['comb_sig'] < filt_val]
    else:
        return df[df['comb_sig'] > filt_val]

#export
def nm_to_px(df, px_size_zyx=[100.,100.,100.]):
    
    if df is None: return None
    df_corr = df.copy()
    df_corr['x'] /= px_size_zyx[2]
    df_corr['y'] /= px_size_zyx[1]
    df_corr['z'] /= px_size_zyx[0]
    return df_corr

#export
def px_to_nm(df, px_size_zyx=[100.,100.,100.]):
    
    df_corr = df.copy()
    df_corr['x'] *= px_size_zyx[2]
    df_corr['y'] *= px_size_zyx[1]
    df_corr['z'] *= px_size_zyx[0]
    return df_corr

#export
def cat_emitter_dfs(df_list, n_frames=1):
    
    if isinstance(n_frames, int):
        n_frames = [n_frames for _ in df_list]
    cum_frames = np.cumsum(n_frames)
    ret_df = df_list[0]
    for df, n_f in zip(df_list[1:], cum_frames[:-1]):
        dfc = df.copy()
        if len(ret_df):
            dfc['frame_idx'] += n_f
            dfc['loc_idx'] += ret_df['loc_idx'].values[-1] + 1
        ret_df = pd.concat([ret_df, dfc], ignore_index=True)
    return ret_df

def append_emitter_df(df1, df2):
    
    if not len(df1):
        return df2
    
    dfc = df2.copy()
    dfc['frame_idx'] += df1['frame_idx'].max() + 1
    dfc['loc_idx'] += df1['loc_idx'].max() + 1    
    
    return pd.concat([df1, dfc], ignore_index=True)

#export
def crop_df(df, fzyx_sl=np.s_[:,:,:,:], shift=True, px_size_zyx=[1.,1.,1.], keys=['frame_idx','z','y','x']):
    
    px_size_zyx = list(px_size_zyx)
    df_crop = df.copy()
    for sl, key, px_s in zip(fzyx_sl, keys, [1] + px_size_zyx):
        if sl.start:
            df_crop = df_crop[df_crop[key] >= px_s*sl.start]
        if sl.stop:
            df_crop = df_crop[df_crop[key] < px_s*sl.stop]
        if (shift) & (sl.start is not None):
            df_crop[key] -= px_s*sl.start
        
    return df_crop

def exclude_borders(df, img_size, px_size_zyx=[1.,1.,1.], border_size_zyx=[1000,400,400]):
    
    zm, ym, xm = img_size[-3:]
    
    sl = np.s_[:,border_size_zyx[0]:zm*px_size_zyx[0]-border_size_zyx[0],
                 border_size_zyx[1]:ym*px_size_zyx[1]-border_size_zyx[1],
                 border_size_zyx[2]:xm*px_size_zyx[2]-border_size_zyx[2]
              ]
    
    return crop_df(df, sl, px_size_zyx=[1.,1.,1.], shift=False)

def get_n_locs(df):
    counts = []
    vcounts = pred_df['frame_idx'].value_counts()
    for i in range(pred_df['frame_idx'].max()+1):
        if i in vcounts.index:
            counts.append(vcounts[i])
    return counts

def sel_int_ch(res_df, codebook):
    
    int_m = [f'int_{i}' for i in range(codebook.shape[1])]
    int_s = [f'int_sig_{i}' for i in range(codebook.shape[1])]
    
    int_arr = res_df.loc[:,int_m].values
    int_sig = res_df.loc[:,int_s].values
    
    int_arr = int_arr[codebook[res_df['code_inds'].values].nonzero()].reshape([int_arr.shape[0], -1])
    int_sig = int_sig[codebook[res_df['code_inds'].values].nonzero()].reshape([int_sig.shape[0], -1])
    
#     print(int_arr.shape)
    
    ret_df = res_df.drop(columns=int_m)
    ret_df = ret_df.drop(columns=int_s)
    
    ret_df[int_m[:4]] = int_arr
    ret_df[int_s[:4]] = int_sig
    
    ret_df['tot_int'] = int_arr.sum(1)
    ret_df['tot_int_sig'] = int_sig.sum(1)

    return ret_df

def zero_int_ch(res_df, codebook):
    
    int_m = [f'int_{i}' for i in range(codebook.shape[1])]
    int_s = [f'int_sig_{i}' for i in range(codebook.shape[1])]
    
    int_arr = res_df.loc[:,int_m].values
    int_sig = res_df.loc[:,int_s].values
    
    int_arr[codebook[res_df['code_inds'].values] == 0] = 0.
    int_sig[codebook[res_df['code_inds'].values] == 0] = 0.
    
    
    ret_df = res_df.drop(columns=int_m)
    ret_df = ret_df.drop(columns=int_s)
    
    ret_df[int_m] = int_arr
    ret_df[int_s] = int_sig

    return ret_df

In [ ]:
# export
from decode_fish.funcs.evaluation import matching
def get_peaks(image,threshold=500,min_distance=20):
    """Peak finding functions. Provides position estimate for bead stacks that are used as initialization for PSF fitting.
    
    Parameters
    ----------
    image: 2D array
        Single bead recording
    threshold: float
        Initial threshold to identify pixel that are considered as possible peaks. 
    min_distance: float
        Minimal distance between two peaks in pixels
        
    Returns
    -------
    peaks: array
        Array of x,y peak positions
    """            
    peaks = []
    t_img = np.where(image>threshold,image,0)
    inds = t_img.nonzero()
    vals = t_img[inds]
    inds_yx = [[y,x] for _,y,x in sorted(zip(vals,inds[0],inds[1]))][::-1]    
    
    while len(inds_yx) > 0:
        
        valid = True
        yx = inds_yx[0]
        y,x = yx
        inds_yx.remove(yx)
        
        for pyx in peaks:
            if np.sqrt((y-pyx[0])**2 + (x-pyx[1])**2) < min_distance:
                valid = False
                break
        if valid:       
            peaks.append(yx)
            
    peak_df = DF(np.array(peaks)[:,::-1], columns=['x','y'])
    peak_df['frame_idx'] = 0
    peak_df['z'] = 0
    peak_df['loc_idx'] = np.arange(len(peak_df))
            
    return peak_df

def remove_fids(res_df, fid_df, tolerance=1000):

    res_c = DF.copy(res_df)
    frames = res_c['frame_idx'].values + 0
    res_c['frame_idx'] = 0
    _, match_fid, _ = matching(fid_df, res_c, tolerance=tolerance, allow_multiple_matches=True, match_genes=False,print_res=False)
    inds = ~res_c['loc_idx'].isin(match_fid['loc_idx_pred'])
    res_clean = res_c.loc[inds]
    res_clean['frame_idx'] = frames[inds]
    
    return res_clean

#export
def remove_doublets(pred_df, tolerance=300):
    _, matches, _ = matching(pred_df, pred_df, tolerance=tolerance, match_genes=True, allow_multiple_matches=True, self_match=True, print_res=False)
    return pred_df.loc[~pred_df['loc_idx'].isin(matches['loc_idx_pred'])]

In [ ]:
!nbdev_build_lib

Converted 00_models.ipynb.
Converted 01_psf.ipynb.
Converted 02_microscope.ipynb.
Converted 02b_place_psfs.ipynb.
Converted 03_noise.ipynb.
Converted 04_pointsource.ipynb.
Converted 05_gmm_loss.ipynb.
Converted 06_plotting.ipynb.
Converted 07_file_io.ipynb.
Converted 08_dataset.ipynb.
Converted 09_output_trafo.ipynb.
Converted 10_evaluation.ipynb.
Converted 11_emitter_io.ipynb.
Converted 12_utils.ipynb.
Converted 13_train.ipynb.
Converted 15_fit_psf.ipynb.
Converted 16_visualization.ipynb.
Converted 17_eval_routines.ipynb.
Converted 18_predict_funcs.ipynb.
Converted 19_MERFISH_routines.ipynb.
Converted 22_MERFISH_codenet.ipynb.
Converted 23_MERFISH_comparison.ipynb.
Converted 24_exp_specific.ipynb.
Converted 25_ensembling.ipynb.
Converted 26_gen_train.ipynb.
Converted 27_testtime_rescale.ipynb.
Converted index.ipynb.
